# I535 - Final Project - Data Processing
By Binh Bui

In [ ]:
#pip install google-cloud-bigquery

In [9]:
import requests
import zipfile
import io
import pandas as pd
from google.cloud import bigquery

## 5500 dataset

In [197]:
# Change to the year you want to download and process
year = '2021'

### Data loading

In [199]:
# Download the ZIP file from the URL
url = 'https://askebsa.dol.gov/FOIA Files/' + year + '/All/F_5500_' + year + '_All.zip' 
response = requests.get(url)

# Open the ZIP file in memory
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# List the files in the ZIP
print(zip_file.namelist()) 

# Extract and read the CSV file into a DataFrame
with zip_file.open('f_5500_' + year + '_all.csv') as csvfile:
    df = pd.read_csv(csvfile, usecols=['ACK_ID','FORM_PLAN_YEAR_BEGIN_DATE','FORM_TAX_PRD','PLAN_NAME','PLAN_EFF_DATE','SPONSOR_DFE_NAME',
                                       'SPONS_DFE_MAIL_US_ADDRESS1','SPONS_DFE_MAIL_US_ADDRESS2','SPONS_DFE_MAIL_US_CITY','SPONS_DFE_MAIL_US_STATE',
                                       'SPONS_DFE_MAIL_US_ZIP','SPONS_DFE_EIN','SPONS_DFE_PHONE_NUM','BUSINESS_CODE',
                                       'TOT_ACTIVE_PARTCP_CNT','PARTCP_ACCOUNT_BAL_CNT'])

print(df.head(1))

['f_5500_2022_all.csv', 'f_5500_2022_all_layout.txt']
                           ACK_ID FORM_PLAN_YEAR_BEGIN_DATE FORM_TAX_PRD  \
0  20230130135522NAL0016717649001                2022-01-01   2022-04-30   

                                     PLAN_NAME PLAN_EFF_DATE  \
0  MCLAURIN & ASSOCIATES, INC. RETIREMENT PLAN    2013-11-08   

              SPONSOR_DFE_NAME SPONS_DFE_MAIL_US_ADDRESS1  \
0  MCLAURIN & ASSOCIATES, INC.         9015 FURROW AVENUE   

  SPONS_DFE_MAIL_US_ADDRESS2 SPONS_DFE_MAIL_US_CITY SPONS_DFE_MAIL_US_STATE  \
0                        NaN          ELLICOTT CITY                      MD   

   SPONS_DFE_MAIL_US_ZIP  SPONS_DFE_EIN  SPONS_DFE_PHONE_NUM  BUSINESS_CODE  \
0                21042.0      464074696         4.432444e+09       541513.0   

   TOT_ACTIVE_PARTCP_CNT  PARTCP_ACCOUNT_BAL_CNT  
0                    0.0                     0.0  


In [200]:
df.shape

(258755, 16)

### Cleaning
The plans in Form 5500 dataset are plans that have from 100 participant and above, so plans that have total active participant less than 2 and total participant that have balance less than 100 are filtered out. Only plans that have FORM_PLAN_YEAR_BEGIN_DATE and FORM_TAX_PRD within the specific year are selected. For example, if the year of 2021 is in the processing, plans that have FORM_PLAN_YEAR_BEGIN_DATE in 2020-10-24 should be filtered out.  

In [201]:
df['FORM_PLAN_YEAR_BEGIN_DATE'] = pd.to_datetime(df['FORM_PLAN_YEAR_BEGIN_DATE'], errors='coerce') # This will set invalid dates to NaT (not a time)
df['FORM_TAX_PRD'] = pd.to_datetime(df['FORM_TAX_PRD'], errors='coerce') # This will set invalid dates to NaT (not a time)

In [202]:
df_filtered = df[(df['TOT_ACTIVE_PARTCP_CNT']>=2)&(df['PARTCP_ACCOUNT_BAL_CNT']>=100)&
                (df['FORM_PLAN_YEAR_BEGIN_DATE']>=pd.to_datetime(year + '-01-01'))&(df['FORM_TAX_PRD']<=pd.to_datetime(year + '-12-31'))]
df_filtered.shape

(68814, 16)

In [203]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68814 entries, 54 to 258754
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ACK_ID                      68814 non-null  object        
 1   FORM_PLAN_YEAR_BEGIN_DATE   68814 non-null  datetime64[ns]
 2   FORM_TAX_PRD                68814 non-null  datetime64[ns]
 3   PLAN_NAME                   68814 non-null  object        
 4   PLAN_EFF_DATE               68814 non-null  object        
 5   SPONSOR_DFE_NAME            68814 non-null  object        
 6   SPONS_DFE_MAIL_US_ADDRESS1  68743 non-null  object        
 7   SPONS_DFE_MAIL_US_ADDRESS2  14014 non-null  object        
 8   SPONS_DFE_MAIL_US_CITY      68743 non-null  object        
 9   SPONS_DFE_MAIL_US_STATE     68743 non-null  object        
 10  SPONS_DFE_MAIL_US_ZIP       68743 non-null  float64       
 11  SPONS_DFE_EIN               68814 non-null  int64        

In [209]:
df_filtered = df_filtered[['ACK_ID','TOT_ACTIVE_PARTCP_CNT']]

In [211]:
# Check duplicates base on ACK_ID
duplicate = df_filtered[df_filtered.duplicated('ACK_ID')]
duplicate.shape

(0, 2)

## Schedule H

### Data loading

In [213]:
# Download the ZIP file from the URL
url = 'https://askebsa.dol.gov/FOIA Files/'+ year +'/All/F_SCH_H_'+ year +'_All.zip' 
response = requests.get(url)

# Open the ZIP file in memory
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

# List the files in the ZIP
print(zip_file.namelist()) 

# Extract and read the CSV file into a DataFrame
with zip_file.open('F_SCH_H_'+ year +'_all.csv') as csvfile:
    df_H = pd.read_csv(csvfile, usecols=['ACK_ID','SCH_H_PLAN_YEAR_BEGIN_DATE','SCH_H_TAX_PRD','TOT_ASSETS_EOY_AMT','PARTICIPANT_CONTRIB_AMT'])

print(df_H.head(1))

['F_SCH_H_2022_all.csv', 'F_SCH_H_2022_all_layout.txt']
                           ACK_ID SCH_H_PLAN_YEAR_BEGIN_DATE SCH_H_TAX_PRD  \
0  20230628134101NAL0011857794001                 2022-01-01    2022-12-31   

   TOT_ASSETS_EOY_AMT  PARTICIPANT_CONTRIB_AMT  
0           5709020.0                 482943.0  


In [214]:
df_H.shape

(112957, 5)

### Cleaning
The plans that have TOT_ASSETS_EOY_AMT greater than or equal to $100,000 and PARTICIPANT_CONTRIB_AMT grater than or equal than $10,000 are selected. Only plans that have SCH_H_FORM_PLAN_YEAR_BEGIN_DATE and SCH_H_FORM_TAX_PRD within the specific year are selected. For example, if the year of 2021 is in the processing, plans that have SCH_H_FORM_PLAN_YEAR_BEGIN_DATE in 2020-10-24 should be filtered out.  

In [217]:
df_H['SCH_H_PLAN_YEAR_BEGIN_DATE'] = pd.to_datetime(df_H['SCH_H_PLAN_YEAR_BEGIN_DATE'], errors='coerce') # This will set invalid dates to NaT
df_H['SCH_H_TAX_PRD'] = pd.to_datetime(df_H['SCH_H_TAX_PRD'], errors='coerce') # This will set invalid dates to NaT

In [219]:
df_H_filtered = df_H[(df_H['TOT_ASSETS_EOY_AMT']>=100000)&(df_H['PARTICIPANT_CONTRIB_AMT']>=10000)&
                    (df_H['SCH_H_PLAN_YEAR_BEGIN_DATE']>=pd.to_datetime(year +'-01-01'))&(df_H['SCH_H_TAX_PRD']<=pd.to_datetime(year +'-12-31'))]
df_H_filtered.shape

(78717, 5)

In [221]:
df_H_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78717 entries, 0 to 112952
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ACK_ID                      78717 non-null  object        
 1   SCH_H_PLAN_YEAR_BEGIN_DATE  78717 non-null  datetime64[ns]
 2   SCH_H_TAX_PRD               78717 non-null  datetime64[ns]
 3   TOT_ASSETS_EOY_AMT          78717 non-null  float64       
 4   PARTICIPANT_CONTRIB_AMT     78717 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 3.6+ MB


In [223]:
df_H_filtered = df_H_filtered[['ACK_ID','TOT_ASSETS_EOY_AMT','PARTICIPANT_CONTRIB_AMT']]

In [225]:
# Check duplicates base on ACK_ID
duplicate = df_H_filtered[df_H_filtered.duplicated('ACK_ID')]
duplicate.shape

(0, 3)

In [227]:
df_cleaned = pd.merge(df_filtered, df_H_filtered, on=['ACK_ID'], how='inner')
df_cleaned.shape

(63900, 4)

In [229]:
df_cleaned.head()

,ACK_ID,TOT_ACTIVE_PARTCP_CNT,TOT_ASSETS_EOY_AMT,PARTICIPANT_CONTRIB_AMT
0,20230613130702NAL0071334002001,292.0,1.686742e+06,740875.0
1,20230613130713NAL0036130113001,180.0,3.148279e+06,208155.0
2,20230615143657NAL0076595522001,5760.0,1.211775e+09,42809662.0
3,20230615143846NAL0025158099001,163.0,8.377090e+05,233074.0
4,20230615150417NAL0025180195001,304.0,1.317914e+07,1081434.0


## Computing Total Asset, Total Active Participant, and Average Paticipant Contribution

In [231]:
total_assets = str(sum(df_cleaned['TOT_ASSETS_EOY_AMT']))
total_active_participants = str(sum(df_cleaned['TOT_ACTIVE_PARTCP_CNT'])).split('.')[0]
avg_participant_contribution = str(sum(df_cleaned['PARTICIPANT_CONTRIB_AMT'])/sum(df_cleaned['TOT_ACTIVE_PARTCP_CNT']))

## Insert data into BigQuery

In [ ]:
client = bigquery.Client.from_service_account_json('sp24-i535-binhbui-5500database-e5cb7de739da.json')

In [233]:
query = """INSERT INTO `sp24-i535-binhbui-5500database.5500_Database.BigPlan`
            VALUES ("""+year+","+total_assets+","+total_active_participants+","+avg_participant_contribution+")"
client.query(query)

QueryJob<project=sp24-i535-binhbui-5500database, location=US, id=a0655990-71d6-4db6-be41-b1c3eed11f25>